In [6]:
!pip install packaging==23.2

   ---------------------------------------- 0.0/53.0 kB ? eta -:--:--
   --------------- ------------------------ 20.5/53.0 kB 330.3 kB/s eta 0:00:01
   ---------------------------------------- 53.0/53.0 kB 678.9 kB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 23.0
    Uninstalling packaging-23.0:
      Successfully uninstalled packaging-23.0


In [7]:
!pip install langchain langchain-ollama openai python-dotenv

In [8]:
import langchain
import openai
import streamlit

In [10]:
!pip install langchain-community

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB 9.3 MB/s eta 0:00:01
   ------------------ --------------------- 1.1/2.5 MB 14.6 MB/s eta 0:00:01
   ------------------------------- -------- 2.0/2.5 MB 15.8 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 16.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/44.4 kB ? eta -:--:--
   ---------------------------------------- 44.4/44.4 kB ? eta 0:00:00
   ---------------------------------------- 0.0/50.9 kB ? eta -:--:--
   ---------------------------------------- 50.9/50.9 kB 2.7 MB/s eta 0:00:00


In [13]:
!pip install langchain-ollama
!pip install langchain-text-splitters

In [27]:
import os
import requests
from textwrap import dedent
from IPython.display import Markdown

# Initialize Ollama
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [28]:
# Configure Ollama model with the local URL
llm = OllamaLLM(model="llama2", api_url="http://localhost:11434")

In [29]:
# Store your API key for Tavily securely
os.environ["TAVILY_API_KEY"] = "your-tavily-api-key-here"  # Make sure this key is correct!

# Function to interact with Tavily API
def tavily_search(query):
    headers = {
        "Authorization": f"Bearer {os.getenv('TAVILY_API_KEY')}"
    }
    params = {
        "query": query,
        "search_depth": "advanced",
        "include_answers": "true",
        "include_raw_content": "false",
        "max_results": 5
    }
    response = requests.post("https://api.tavily.com/search", headers=headers, json=params)
    data = response.json()

    print("Tavily API Response:", data)  # Log the response for debugging

    if 'results' in data:
        return data["results"]
    else:
        return []

# Function to format the research results
def research_agent(query):
    results = tavily_search(query)
    summaries = []
    for item in results:
        summaries.append(f"{item['title']}: {item['content'][:300]}... ({item['url']})")
    return "\n".join(summaries)


In [33]:
# Configure Ollama
llm = Ollama(model="llama2", temperature=0.7)

# Prompt template with clear formatting instructions
template = """
You are an expert research analyst. Based on the following research results:
{research_results}
Please provide a well-structured response with these sections:
1. Key Findings (bullet points)
2. Detailed Analysis (2-3 paragraphs)
3. Top References (formatted as [Title](URL))
Format the response in Markdown for better readability.
"""

prompt = PromptTemplate(
    input_variables=["research_results"],
    template=template
)
chain = LLMChain(llm=llm, prompt=prompt)

In [34]:
def tavily_search(query):
    headers = {"Authorization": f"Bearer {os.getenv('TAVILY_API_KEY')}"}
    params = {
        "query": query,
        "search_depth": "advanced",
        "include_answers": True,
        "include_raw_content": False,
        "max_results": 5
    }
    response = requests.post("https://api.tavily.com/search", headers=headers, json=params)
    return response.json()

In [35]:

def format_research_results(results):
    formatted = []
    
    # Add Tavily's answer if available
    if 'answer' in results and results['answer']:
        formatted.append(f"## Tavily Summary\n{results['answer']}\n")
    
    # Format individual results
    for item in results.get('results', []):
        formatted.append(dedent(f"""
        - **Title**: {item.get('title', 'N/A')}
          **URL**: [{item.get('url', 'N/A')}]({item.get('url', 'N/A')})
          **Content**: {item.get('content', 'N/A')[:200]}...
          **Relevance Score**: {item.get('score', 0):.2f}
        """))
    return "\n".join(formatted)

In [36]:
def complete_research(query):
    """
    Complete research and return results as Markdown
    """
    # Get and format research
    raw_results = tavily_search(query)
    research_results = format_research_results(raw_results)
    
    # Generate final answer
    response = chain.invoke({"research_results": research_results})
    
    # Post-process for consistent formatting
    formatted_response = (
        response['text']
        .replace('•', '-')  # Standardize bullet points
        .replace('\n\n\n', '\n\n')  # Remove excessive newlines but keep paragraph breaks
    )
    
    # Add header and query info
    final_output = f"# Research Results: {query}\n\n{formatted_response}"
    
    # Return as Markdown for display
    return Markdown(final_output)

# Usage example
query = "What are the key trends in AI technology?"
complete_research(query)

# Research Results: What are the key trends in AI technology?

Key Findings:

* AI trends in 2025 will focus on improving decision-making through more accessible and accurate data-driven decision-making tools.
* Companies are increasingly looking for proven results from generative AI, rather than early-stage prototypes.
* Artificial intelligence (AI) has taken the technology industry by storm, with 73% of US companies using AI in some capacity.
* Eight important AI trends to watch in 2025 include:
	+ Genetic AI
	+ Agents and bots
	+ Edge AI
	+ Explainable AI
	+ Human-AI collaboration
	+ Sustainability and environmental impact
	+ Ethics and governance
	+ Quantum AI

Detailed Analysis:
The articles provide valuable insights into the current state of AI technology and its future trajectory. The trend towards improving decision-making through more accessible and accurate data-driven decision-making tools is likely to continue, as companies seek to harness the power of AI to make better decisions. The increased demand for proven results from generative AI also highlights the growing importance of accuracy and reliability in AI technology.

The articles also provide a comprehensive overview of the current state of AI technology, including its applications, challenges, and future directions. The focus on sustainability and environmental impact underscores the need for AI technology to be developed and used responsibly, with consideration for its broader social implications.

Top References:

[Title] 14 AI Trends Transforming Tech in 2025 and Beyond | DigitalOcean
[URL] https://www.digitalocean.com/resources/articles/ai-trends

[Title] 8 AI and machine learning trends to watch in 2025 | TechTarget
[URL] https://www.techtarget.com/searchenterpriseai/tip/9-top-AI-and-machine-learning-trends

[Title] Top 5 AI Trends to Watch in 2025 - Coursera
[URL] https://www.coursera.org/articles/ai-trends

[Title] AI Trends for 2025 - YouTube
[URL] https://www.youtube.com/watch?v=5zuF4Ys1eAw

[Title] 6 AI trends you'll see more of in 2025 - Microsoft News
[URL] https://news.microsoft.com/source/features/ai/6-ai-trends-youll-see-more-of-in-2025/